In [11]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from keras import regularizers
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.utils import plot_model
import graphviz, pydot, pydotplus
from sklearn.model_selection import train_test_split
import tensorflow as tf
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Обучение линейной модели, получение бейзлайна

берем данные

In [12]:
measurements = []
classifications = []

root_folder = "drive/MyDrive/h"
healthy = []
for root, dirs, files in os.walk(root_folder):  
    for filename in files:
        healthy.append(filename)
count_healthy = 0

for i, name in enumerate(healthy):
  if i>=3:
    break
  df = pd.read_csv("drive/MyDrive/h/" + name, sep='\t',skiprows=[0],
  header=None, names=['X', 'Y', 'Wave', 'Intensity'])
  for i in range(len(df)//1015):
    measurements.append(df[['Intensity']][i*1015:(i+1)*1015].to_numpy())
    classifications.append([1,0])
    count_healthy += 1



root_folder = "drive/MyDrive/s"
sick = []
for root, dirs, files in os.walk(root_folder):  
    for filename in files:
        sick.append(filename)

count_sick = 0

for i, name in enumerate(sick):
  if i>=3:
    break
  df = pd.read_csv("drive/MyDrive/s/" + name, sep='\t',skiprows=[0],
  header=None, names=['X', 'Y', 'Wave', 'Intensity'])
  for i in range(len(df)//1015):
    measurements.append(df[['Intensity']][i*1015:(i+1)*1015].to_numpy())
    classifications.append([0,1])
    count_sick += 1

In [13]:
X = np.asarray(measurements)
y = np.asarray(classifications)
X = X.reshape(X.shape[0], 1015)
X.shape

(2640, 1015)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=228)
print(f'Total amount of train measurements: {X_train.shape}')
print(f'Total amount of train labels: {y_train.shape}')
print(f'Total amount of test measurements: {X_test.shape}')
print(f'Total amount of test labels: {y_test.shape}')
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

Total amount of train measurements: (1980, 1015)
Total amount of train labels: (1980, 2)
Total amount of test measurements: (660, 1015)
Total amount of test labels: (660, 2)


данные готовы 

переходим к обучению

In [7]:
from sklearn.metrics import f1_score, accuracy_score


In [15]:
from sklearn.svm import SVC
m = SVC(kernel='linear')
m.fit(X_train, y_train_labels) 

print(f'SVC F1 Score {f1_score(y_test_labels, m.predict(X_test))}')
print(f'SVC Accuracy {accuracy_score(y_test_labels, m.predict(X_test))}')

SVC F1 Score 0.9598930481283423
SVC Accuracy 0.9545454545454546


теперь посмотрим результаты на других данных

---



In [16]:
import pickle

pickle.dump(m, open('svc_model_base', 'wb'))
